In [1]:
dataset = qml.data.load('qchem', molname="H2", basis=ST)[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)

NameError: name 'qml' is not defined

In [8]:
from pennylane import qchem
symbols, coordinates = qchem.read_structure("H2O.xyz")

In [9]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

Number of qubits: 14
Qubit Hamiltonian
  (-46.43203980592803) [I0]
+ (0.7743409816426627) [Z10]
+ (0.7743409816426627) [Z11]
+ (0.8085591788250823) [Z12]
+ (0.8085591788250825) [Z13]
+ (1.2103142554658215) [Z5]
+ (1.2103142554658217) [Z4]
+ (1.3161884200290686) [Z6]
+ (1.316188420029069) [Z7]
+ (1.370451700309563) [Z9]
+ (1.3704517003095635) [Z8]
+ (1.6548866623445804) [Z2]
+ (1.6548866623445804) [Z3]
+ (12.412875617544046) [Z0]
+ (12.412875617544048) [Z1]
+ (-0.10437515218162376) [Y0 Y2]
+ (-0.10437515218162376) [X0 X2]
+ (-0.0033038081016212397) [Y1 Y3]
+ (-0.0033038081016212397) [X1 X3]
+ (0.11150862218070981) [Z10 Z12]
+ (0.11150862218070981) [Z11 Z13]
+ (0.11475504456567351) [Z4 Z12]
+ (0.11475504456567351) [Z5 Z13]
+ (0.11993628274078363) [Z6 Z10]
+ (0.11993628274078363) [Z7 Z11]
+ (0.12352309405647796) [Z4 Z10]
+ (0.12352309405647796) [Z5 Z11]
+ (0.12433325269360518) [Z2 Z4]
+ (0.12433325269360518) [Z3 Z5]
+ (0.1267947726852547) [Z2 Z10]
+ (0.1267947726852547) [Z3 Z11]
+ (0.1360

In [10]:
import pennylane as qml
from pennylane import qchem

# Hartree-Fock state
hf = qml.qchem.hf_state(electrons=7,orbitals=10)

In [12]:
from pennylane import numpy as np

# atomic symbols defining the molecule
symbols = ['H','H', 'O']

# list to store energies
energies = []

# set up a loop to change bond length
r_range = np.arange(0.5, 5.0, 0.25)

# keeps track of points in the potential energy surface
pes_point = 0

In [15]:
pip install openfermionpyscf


  Using cached openfermionpyscf-0.5-py3-none-any.whl.metadata (6.9 kB)
  Using cached pyscf-2.5.0.tar.gz (8.7 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached iniconfig-2.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached pluggy-1.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached exceptiongroup-1.2.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
Using cached openfermionpyscf-0.5-py3-none-any.whl (16 kB)
   ---------------------------------------- 0.0/339.2 kB ? eta -:--:--
   --------------------------------------  337.9/339.2 kB 10.2 MB/s eta 0:00:01
   ---------------------------------------- 339.2/339.2 kB 5.2 MB/s eta 0:00:00
Using cached exceptiongroup-1.2.1-py3-none-any.whl (16 kB)
Using cached pluggy-1.5.0-py3-none-any.whl (20 kB)
Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
  Running setup.

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_py
      cmake -SC:\Users\DELL\AppData\Local\Temp\pip-install-fe8v0n3i\pyscf_a9dfbd7798b7417f8387d0b5853658f1\pyscf\lib -Bbuild\temp.win-amd64
      error: command 'cmake' failed: None
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyscf
ERROR: Could not build wheels for pyscf, which is required to install pyproject.toml-based projects


In [ ]:
for r in r_range:
    # Change only the z coordinate of one atom
    coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r, 0.0, 0.0, -r])

    # Obtain the qubit Hamiltonian
    H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method='pyscf')

    # define the device, optimizer and circuit
    dev = qml.device("default.qubit", wires=qubits)
    opt = qml.GradientDescentOptimizer(stepsize=0.4)

    @qml.qnode(dev, interface='autograd')
    def circuit(parameters):
        # Prepare the HF state: |1100>
        qml.BasisState(hf, wires=range(qubits))
        qml.DoubleExcitation(parameters[0], wires=[0, 1, 2, 3])
        qml.SingleExcitation(parameters[1], wires=[0, 2])
        qml.SingleExcitation(parameters[2], wires=[1, 3])

        return qml.expval(H)  # we are interested in minimizing this expectation value

    # initialize the gate parameters
    params = np.zeros(3, requires_grad=True)

    # initialize with converged parameters from previous point
    if pes_point > 0:
        params = params_old

    prev_energy = 0.0
    for n in range(50):
        # perform optimization step
        params, energy = opt.step_and_cost(circuit, params)

        if np.abs(energy - prev_energy) < 1e-6:
            break
        prev_energy = energy

    # store the converged parameters
    params_old = params
    pes_point = pes_point + 1

    energies.append(energy)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(r_range, energies)

ax.set(
    xlabel="Bond length (Bohr)",
    ylabel="Total energy (Hartree)",
    title="Potential energy surface for H$_2$ dissociation",
)
ax.grid()
plt.show()